In [ ]:
import numpy as np
import pandas as pd
from mido import MidiFile
from collections import namedtuple

In [ ]:
midi_dir = "../midi/"
TimedNote = namedtuple('TimedNote', ['note', 'start_tick', 'end_tick'])
CustomMessage = namedtuple('CustomMessage', ['start_tick', 'note', 'on'])

In [ ]:
"""
Structure for keeping track of all notes playing at the same time
"""
class ActiveNotes:
    def __init__(self):
        self.cur_tick = 0
        self.currently_playing = np.zeros([128], dtype='bool')
        # Array to prevent adding the same highest note twice
        # (f.e. some note gets turned off and some note which has been added to the melody already
        # might still be playing and gets evaluated as highest note again)
        self.already_in_melody = np.zeros([128], dtype='bool')

    def note_on(self, note):
        self.currently_playing[note] = True

    def note_off(self, note):
        self.currently_playing[note] = False
        self.already_in_melody[note] = False

    """
    Return highest note and ticks when it started/stopped
    delta_tick: time since latest MIDI Message
    """
    def get_highest_note(self, delta_tick):
        previous_tick = self.cur_tick
        self.cur_tick = self.cur_tick + delta_tick
        for i in range(127, -1, -1):
            # Found the highest note that's currently playing
            if self.currently_playing[i]:
                # It hasn't been added to melody yet; add it
                if not self.already_in_melody[i]:
                    self.already_in_melody[i] = True
                    return TimedNote(i, previous_tick, self.cur_tick)

                # Has been added to melody; stop the search for highest note
                else:
                    break

        return TimedNote(-1, previous_tick, self.cur_tick)


In [ ]:
def get_highest_melody(file, tracks_to_merge, tick_ignore):
    mid = MidiFile(file)
    highest_melodies = []
    merged_melody = []

    # Process all tracks separately first
    for track_no in tracks_to_merge:
        an = ActiveNotes()
        highest_melody = []
        additive_tick = 0
        for msg in mid.tracks[track_no]:
            if msg.type in ['note_on', 'note_off']:
                # Not instant; some time has passed => evaluate previous highest note
                tick = msg.time + additive_tick
                additive_tick = 0
                if tick != 0:
                    res = an.get_highest_note(tick)
                    # Filter out cases when nothing is playing
                    if res.note != -1:
                        # highest_melody.append(res)
                        # Add turn on/turn off messages
                        highest_melody.append(CustomMessage(res.start_tick, res.note, 1))
                        highest_melody.append(CustomMessage(res.end_tick, res.note, 0))

                if msg.velocity != 0 and msg.type == 'note_on':
                    an.note_on(msg.note)
                else:
                    an.note_off(msg.note)

            # Some messages just change time for no FUCKING REASON
            else:
                additive_tick = additive_tick + msg.time

            # if msg.type in ['set_tempo', 'control_change', 'time_signature']:
            #    additive_tick = additive_tick + msg.time

        highest_melodies.append(highest_melody)

    # Merge all highest melody messages into one list
    mega_list = []
    for i in highest_melodies:
        mega_list = mega_list + i

    # Sort all messages by starting tick
    mega_list.sort(key=lambda custom_msg: custom_msg.start_tick)
    an = ActiveNotes()
    cur_tick = mega_list[0][0]

    for custom_msg in mega_list:
        # Treat a very small time difference between messages as if the events happened simultaneously
        # So that the higher note gets judged correctly
        if abs(custom_msg.start_tick - cur_tick) > tick_ignore:
            res = an.get_highest_note(0)
            if res.note != -1:
                merged_melody.append(res.note)

        if custom_msg.on:
            an.note_on(custom_msg.note)

        else:
            an.note_off(custom_msg.note)

        cur_tick = custom_msg.start_tick


    return merged_melody, mega_list


In [ ]:
res = get_highest_melody("../backend/midi/bach_846.mid", [1, 2], 10)
print(res[0])

In [ ]:
# Load a midi file
# "../backend/midi/bach_846.mid"
midd = MidiFile("../upload/untitled.mid")
# Print notes in midi track
for track in midd.tracks:
    print("trck: " + track.name)

print(midd.type)
extracted_melody = get_highest_melody("../upload/untitled.mid", [2], 10)
print(extracted_melody[0])

In [ ]:
for msgg in midd.tracks[2]:
    # Not instant; some time has passed => evaluate previous highest note
    print(msgg)

In [ ]:
def get_metadata(file):
    # Load a midi file
    mid = MidiFile(file)
    track_name = ""
    author = ""
    # First track contains data
    for msg in mid.tracks[0]:
        # Not instant; some time has passed => evaluate previous highest note
        if msg.type == "track_name":
            if track_name == "":
                track_name = msg.name
            else:
                track_name = track_name + ": " + msg.name

        # First text message contains the author
        if msg.type == "text":
            author = msg.text
            break

    # Second and third tracks contain right/left hand
    melody = get_highest_melody(file, [1, 2], 5)[0]
    return {
        "track_name": track_name,
        "author": author,
        "melody": melody
    }

In [ ]:
res = get_metadata("../backend/midi/chpn_op53.mid")
print(res)